# Chiffrement hybride en `Python` avec la librairie `Cryptography`

**TRINÔME:** BRIKI Jawel, CHANTOISEAU Sacha, ZEGBEU Calvin

Le mini-projet comporte 20 fonctions à programmer. Elles ont pratiquement toutes été vues dans les séances précédentes. Vous ajouterez des cellules à l'énoncé pour valider le bon fonctionnement de chacune des fonctions écrites (et dérouler un exemple d'exécution). En l'absence des exemples de validation, la note par exercice sera divisée par deux.

Vous avez à disposition les fichiers `root_CA.crt` et la clé privée correspondante `CA.pem` ainsi que le certificat de ma clé publique `bmartin.crt` (l'énoncé décrit leur usage).

Vous rendrez une archive au format `zip` sur la boîte de dépôt `Moodle` avant le 15 mai 2025 nommée `ICS-Nom1-Nom2.zip` (avec les noms du groupe). Les binômes (ou trinômes) sont autorisés. L'archive comprendra:

- la feuille `jupyter` complétée avec les fonctions et des tests de bon fonctionnement des fonctions;
- un certificat de clé RSA de 1024 bits au format `PEM` (cf. partie **2**) et la clé privée correspondante;
- les deux scripts `Python` demandés à la fin de l'énoncé avec, dans une cellule, les appels à tester par un copier/coller dans un terminal (cf. partie **3**) ou un appel par `os.system()`;
- un petit texte chiffré signé pour le destinataire `bmartin`.

La feuille de calcul remplie correctement avec les tests de bon fonctionnement rapporte 20 points. 

## Importation des librairies

In [ ]:
import zlib, binascii, secrets, os, base64, pickle, datetime
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import padding, rsa, utils
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.x509 import Certificate, DNSName, load_pem_x509_certificate

In [106]:
def gencle(bits:int)->bytes:
    if bits % 8 != 0:
        raise Exception("La taille de la clé doit être un multiple de 8")
    return secrets.token_bytes(bits // 8)

In [107]:
# -- Exemple --
cle_192_bits = gencle(192)
print(f"Clé générée (192 bits): {cle_192_bits.hex()}")

Clé générée (192 bits): 5f6d35dcd646da08631f03960d6be61e8f175d4b695d9ec1


## 1.2 Compression des clairs

La compression est souvent une étape préalable au chiffrement. On utilise la librairie `zlib`.

**Exercice 2.1** Ecrivez une fonction `compresse` qui prend en entrée une chaîne de caractères `utf-8`, la convertit en `bytestream` avec comme type de sortie un `bytestream`.

In [108]:
def compresse(texte:str)-> bytes:
    bytestream = texte.encode("utf-8")
    return zlib.compress(bytestream)

**Exercice 2.2.** Ecrivez La fonction `decompresse` qui prend en entrée le compressé au format `bytestream` et restitue la chaîne originale au format `utf-8`. 

In [109]:
def decompresse(comprime:bytes)->str:
    decompressed_text = zlib.decompress(comprime)
    return decompressed_text.decode("utf-8")

In [110]:
# -- Exemple --
text = "ceci est un exemple"
compressed_text = compresse(text)
print(f"Résultat de la compression: {compressed_text}")
decompressed_text = decompresse(compressed_text)
print(f"Chaîne originale décompressée: {decompressed_text}")

Résultat de la compression: b'x\x9cKNM\xceTH-.Q(\xcdSH\xadH\xcd-\xc8I\x05\x00D\xf3\x07\x14'
Chaîne originale décompressée: ceci est un exemple


## 1.3 Dérivation de clé

La fonction qui permet de dériver une clé a été vue dans le **TD5**. Elle s'inspire directement de la [documentation](https://cryptography.io/en/latest/hazmat/primitives/key-derivation-functions/).

**Exercice 3.** Ecrivez la fonction `derive` qui prend en entrée un secret initial (p.e. de 192 bits) et la taille de la clé dérivée (en bits). Elle retourne la clé dérivée du secret initial de la taille spécifiée en entrée. Les paramètres de la fonction de dérivation sont les premiers 128 bits du secret initial et les 64 bits suivants constituent le sel.

In [111]:
def derive(secret:bytes, bits:int)->bytes:
    if bits % 8 != 0:
        raise ValueError("La taille de la clé doit être un multiple de 8")
    kdf_param = secret[:16]
    salt = secret[16:]
    kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=(bits // 8),
    salt=salt,
    iterations=1_200_000
    )
    return kdf.derive(kdf_param)

In [112]:
## -- Exemple --
secret_initial = gencle(192)
cle_derivee = derive(secret_initial, 256)
print(f"Clé dérivée: {cle_derivee.hex()}")
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=secret_initial[16:],
    iterations=1_200_000
    )

# Si cette ligne ne génère pas d'erreur, la clé est effectivement dérivée du secret initial fourni
kdf.verify(secret_initial[:16], cle_derivee)

Clé dérivée: 4282ba58d329ad5aace83b4afa4b03c26931ff9328a77234728774ad4a864c8d


## 1.4 Fonctions de chiffrement par AES

On écrit les fonctions `encAES` et `decAES` analogues à celles du **TD2** pour chiffrer et déchiffrer un texte par `AES-128-CTR`. L'avantage du mode `CTR` est qu'il n'a pas besoin de bourrage.

**Exercice 4.** Ecrivez la fonction `encAES` qui prend en entrée un texte au format `utf-8` et un secret initial de 192 bits. Elle va successivement compresser le texte avec la fonction `compresse`, calculer une clé de session et un IV dérivés par la fonction `derive` et chiffrer le compressé. Le chiffré sera retourné au format `bytestream`.

In [113]:
def encAES(texte:str, secret:bytes)->bytes:
    compressed_text = compresse(texte)
    
    derived_key = derive(secret, 256)
    session_key = derived_key[:16]
    iv = derived_key[16:]
    
    cipher = Cipher(algorithms.AES128(session_key), modes.CTR(iv))
    encryptor = cipher.encryptor()
    encrypted_data = encryptor.update(compressed_text)

    return encrypted_data

**Exercice 5.** Ecrivez la fonction `decAES` qui inverse le fonctionnement de la fonction de chiffrement et retournera un texte au format `utf-8`.

In [114]:
def decAES(cryptogramme:bytes, secret:bytes)->str:
    derived_key = derive(secret, 256)
    session_key = derived_key[:16]
    iv = derived_key[16:]

    cipher = Cipher(algorithms.AES128(session_key), modes.CTR(iv))
    decryptor = cipher.decryptor()
    decrypted_data = decryptor.update(cryptogramme)

    plaintext = decompresse(decrypted_data)
    return plaintext

In [115]:
# -- Exemple --
secret_initial = gencle(192)
texte = "ceci est un clair"
ciphertext = encAES(texte, secret_initial)
print(f"Texte chiffré par AES-128-CTR: {ciphertext.hex()}")
print(f"Texte déchiffré: {decAES(ciphertext, secret_initial)}")

Texte chiffré par AES-128-CTR: 4ce381857368c8b24c3bc2adf048fa65e7a8246b83511daf4c
Texte déchiffré: ceci est un clair


## 1.5 Génération de clé RSA et enregistrement au format `pem`

Les fonctions pour engendrer, écrire et lire une clé RSA  s'inspirent de celles du **TD3**. 

**Exercice 6.** Ecrivez la fonction `genRSA` qui prend en entrée la taille en bits de la clé RSA à engendrer et qui retourne l'objet de clé privée correspondant.

In [116]:
def genRSA(taille:int)->rsa.RSAPrivateKey:
    return rsa.generate_private_key(
        public_exponent=65537,
        key_size=taille
    )

**Exercice 7.** Ecrivez la fonction `saveRSA` qui prend en entrée un objet de clé RSA (privée ou publique)  et un nom de fichier (d'extension `.pem`). La fonction va enregistrer l'objet de clé au format `PEM` dans le fichier spécifié. La clé privée sera au format `PKCS8` et la publique au format `PKCS1`.

In [117]:
def saveRSA(key, fic_cle:str):
    if isinstance(key, rsa.RSAPrivateKey):
        pem = key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        )
    else:
        pem = key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.PKCS1
        )

    with open(fic_cle, 'wb') as file:
        file.write(pem)

**Exercice 8.** Ecrivez la fonction `readRSA` qui prend comme entrée un nom de fichier (d'extension `.pem`). Elle va lire ce fichier au format `PEM` et reconstruire l'objet `Cryptography` correspondant.

In [118]:
def readRSA(fic_cle:str):
    with open(fic_cle, 'rb') as file:
        pem = file.read()
        if b"PRIVATE" in pem:
            key = serialization.load_pem_private_key(
                pem,
                password=None
            )
        else:
            key = serialization.load_pem_public_key(pem)
        return key

In [119]:
# -- Exemple --
sk = genRSA(2048)
cle = 'ma_cle_privee.pem'
saveRSA(sk, cle)
cle_recue = readRSA(cle)
if isinstance(cle_recue, rsa.RSAPrivateKey):
    print(f'Cle privee.')
else : 
    print(f'Cle publique.')

Cle privee.


## 1.6 Chiffrement et déchiffrement par RSA

Les fonctions `encRSA` et `decRSA` sont analogues à celles écrites dans le **TD3** avec le padding `OAEP`. 

**Exercice 9.** Ecrivez la fonction`encRSA` qui va chiffrer un clair au format `bytestream` avec la clé publique. 

In [120]:
def encRSA(octets:bytes, clepub:rsa.RSAPublicKey)->bytes:
    return clepub.encrypt(
        octets,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

**Exercice 10.** Ecrivez la fonction `decRSA` qui va déchiffrer le chiffré (binaire) avec la clé privée et retourner la suite d'octets initiale. 

In [121]:
def decRSA(octets:bytes, clepriv:rsa.RSAPrivateKey)->bytes:
    return clepriv.decrypt(
        octets,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

In [122]:
## -- Exemple --

entree = "voici un magnifique exemple"
print(f'Chaîne originale: {entree}')
pk = genRSA(2048)
pk = sk.public_key()

compresse_ = compresse(entree)
print (f'Chaine décompressée: {compresse_}')
encode = encRSA(compresse_, pk)
print(f'Chiffré: {encode}')
decode = decRSA(encode, sk)
print(f'Déchiffré: {decode}')
sortie = decompresse(decode)
print (f'Chaine déchiffrée et décompressée: {sortie}')

Chaîne originale: voici un magnifique exemple
Chaine décompressée: b'x\x9c+\xcb\xcfL\xceT(\xcdS\xc8ML\xcf\xcbL\xcb,,MUH\xadH\xcd-\xc8I\x05\x00\x90\xb8\nt'
Chiffré: b"&\xb2P_\x0c\x1b?/K(D\xa6\x1d\xdd\x0c\xdc\x8c$I7\xde\x85\xdc\xdf\x87\xefm\x9du6\xac\xbf\xf6?.\xc0u\x07\xc8\x1e\xef=\x19\xc7\xc3\x8bb|\x7f\x90}P\x86\xba\x08\x96AO935D\xe3*0\xe6eJ\xc05\xd6\x16\xc59'\xbf\xd7\x1e|\xf9>\xab\x1a\xf6\x81.t\x8a\x1c\xda\x8d\x84\x9ar3\x15\x8a\xad\n\x0c\xddS\xb5\x0e\r\xf5+\xea\xec\xbcH\x1e{\xe3A\xa8\xee-)#\xee\x0c\xfd\x06\\\xf7\x9fxsR\x8d\x84h\xee\xf2\xbe\xab\xd8\xfft\xcb\x91\xcb@\xb2\x1c;\x99\xc9\x9c\x86\xcc\xb9r\xbf\xd0\x0b\x915gX7\x0bWY,O\xf3\x1e\x8e\x81\xab\x05\xe3;~\xcb\x1c\xf0\x9b\xe8\x8f\xfd\xa7i\xa8\x90\xc5\xa4\x95;R\xc1\xaf|\x1e\x910\xd4Qi\xeasL\xb1\xea\xecL'\xe6\xec\x1fEL\xd4\x8f\xe7X\xf2\x91\x05\xfb'\xc4'N\\\xb6\n~\x9f\x03\xd5\xde\x95-z\xe1\x14\x18/\xe7\xfa\xd1r\x1fQ\xfe\x8fp\xfd\xe5 \xa2e\x0e"
Déchiffré: b'x\x9c+\xcb\xcfL\xceT(\xcdS\xc8ML\xcf\xcbL\xcb,,MUH\xadH\xcd-\xc8I\x05\x00\x90\xb8\nt

## 1.7 Chiffrement hybride


Ce travail a pour but d'implémenter un chiffrement hybride à la `PGP` pour lequel on va chiffrer par RSA le secret initial utilisé pour chiffrer un message par `AES`. 
<br>


**Exercice 11.** Ecrivez la fonction de chiffrement `chiffre` qui prend en entrée un texte clair et la clé publique du destinataire et qui va fournir la concaténation :

- du secret initial chiffré par la clé publique RSA du destinataire dans une enveloppe digitale;
- le chiffrement du clair par `AES-128-CTR` utilisant le secret initial de 192 bits.

Quelques précisions sur le format du chiffré hybride, au format `bytestream` qui contient:
- la suite de 24 octets du secret initial est chiffrée par RSA avec la clé publique du destinataire. La taille de sortie de cette enveloppe digitale est de 128 octets;
- on concatène ensuite le message chiffré avec le chiffre symétrique.

Ecrit en notation "Alice et Bob", en notant $K$ le secret initial, $pk$ la clé publique du destinataire et $m$ le message, on aura: $$\{K\}_{pk}.\{m\}_K$$
où $.$ dénote l'opération de concaténation.

Pratiquement, ces valeurs seront sérialisées au moyen de la librairie `pickle` et l'ensemble est ensuite converti au format `base64` et retourné à l'utilisateur.


In [123]:
def chiffre(message:str, pk:rsa.RSAPublicKey)->bytes:
    secret = gencle(192)
    encrypted_secret = encRSA(secret, pk)
    ciphertext = encAES(message, secret)
    data = (encrypted_secret, ciphertext)
    pickled = pickle.dumps(data)
    return base64.b64encode(pickled)

**Exercice 12.** Ecrivez la fonction `dechiffre` qui prend en entrée le chiffré hybride au format `base64` et la clé privée. Elle va:

- décoder la suite sérialisée fournie au format `base64` pour retrouver les suites d'octets;
    - le secret initial dans l'enveloppe digitale à déchiffrer pour récupérer le secret initial;
    - le texte chiffré par `AES-128-CTR` à déchiffrer avec le secret initial récupéré.

In [124]:
def dechiffre(chiffre64:bytes, sk:rsa.RSAPrivateKey)->str:
    decoded = base64.b64decode(chiffre64)
    encrypted_secret, ciphertext = pickle.loads(decoded)
    secret = decRSA(encrypted_secret, sk)
    return decAES(ciphertext, secret)

In [125]:
# -- Exemple --
message = "ceci est un clair"
sk = genRSA(2048)
pk = sk.public_key()
chiffre64 = chiffre(message, pk)
print(f"Message chiffré et encodé: {chiffre64}")
decode = dechiffre(chiffre64, sk)
print(f"Message décodé et déchiffré: {decode}")

Message chiffré et encodé: b'gASVJQEAAAAAAABCAAEAACSCWJbdVC3737k1SB1KMDPnh50jd2mSP7RTnM7eRpXZKrmjD4OyRhZZpIvOGW+aammY0X5q0xfA8uFmQ+B7c/aOr/qYJPK0Wqa1iTovlIe5JliOIvyG9B++ezXIK9th9pB36gDCsKyXw+Xh0mFVoYFJTe4zDhfVQN7P0TkHu2UAWzNgpgeeRyxTeg3csF19uqoQTzioUkAH7hT/zgMfwYzOzZN87Bil72yD9/6Ac+LMuUjgArKySXlHz+zSa/gxM6K4fZs1sEyJdDVe0DwtLw0iDggI1QdmlBBgj7VQ90et1TzLA0fxepk1rWCskIpJkwDLvBZLDPJIiBH05PxbXuGUQxmFuvyFdUS4ZALaWnkKMYicyAE+dPnY5vualIaULg=='
Message décodé et déchiffré: ceci est un clair


## 1.8 Signer et vérifier par RSA

Il s'agit maintenant d'ajouter une authentification de l'expéditeur et d'assurer l'intérgrité. On utilise pour cela les fonctions `sigRSA` et `verRSA` utilisant RSA avec le padding `PSS` qui s'inspirent de celles écrite pour DSA au **TD4**.

**Exercice 13.** Ecrivez la fonction `sigRSA` qui va signer un message au format `bytestream` avec la clé privée de l'expéditeur et retourner la signature au format `base64`.

In [126]:
def sigRSA(message:bytes, sk:rsa.RSAPrivateKey)->bytes:
    return sk.sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

**Exercice 14.** Ecrivez la fonction `verRSA` qui utilise la clé publique de l'expéditeur et vérifie que la signature signe bien le message transmis.

In [127]:
def verRSA(message:bytes, signature:bytes, pk:rsa.RSAPublicKey)->str:
    try:
        pk.verify(
            signature,
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception as e:
        print(e)
        return False

In [128]:
# -- Exemple --
message = compresse("Vive la cryptologie!")
signe = sigRSA(message,sk)
if verRSA(message, signe, pk):
    print('La signature signe bien le message transmis.')
else:
    print('La signature ne signe pas le message transmis.')

La signature signe bien le message transmis.


## 1.9 Chiffrement hybride authentifié

Vous pouvez maintenant assurer l'authentification de l'expéditeur au chiffrement hybride et l'intégrité du message. L'expéditeur va signer tout le chiffré et transmettre la sérialisation du message et de sa signature.

**Exercice 15.** Ecrivez la fonction `HencS` qui prend en entrée un texte clair au format `utf-8`, la clé publique du destinataire, la clé privée de l'expéditeur et va:

- appliquer la fonction de chiffrement hybride `chiffre`;
- signer le chiffré obtenu précédemment au format `base64`;
- sérialiser le chiffré et la signature au moyen de la librairie `pickle`

Ecrit en notation "Alice et Bob", en notant $K$ le secret initial, $pkd$ la clé publique du destinataire, $ske$ la clé privée de l'expéditeur et $m$ le message, on aura: $$\{K\}_{pkd}.\{m\}_K.\{\{K\}_{pkd}\{m\}_K\}_{ske}$$
où $.$ dénote l'opération de concaténation.

In [129]:
def HencS(message:str, pk:rsa.RSAPublicKey, sk:rsa.RSAPrivateKey)->bytes:
    chiffre64 = chiffre(message, pk)
    signature = sigRSA(chiffre64, sk)
    data = (chiffre64, signature)
    pickled = pickle.dumps(data)
    return base64.b64encode(pickled)

**Exercice 16.** Ecrivez la fonction `HDecS` qui prend en entrée le message au format `base64`. Elle va:

- décoder le message chiffré et signé pour retrouver la concaténation du chiffré et la signature;
- extraire le chiffré et sa signature;
- appliquer la fonction `verifie`;
- déchiffrer le chiffré et l'afficher.

In [130]:
def HdecS(message64:bytes, pk:rsa.RSAPublicKey, sk:rsa.RSAPrivateKey)->str:
    decoded = base64.b64decode(message64)
    ciphertext, signature = pickle.loads(decoded)
    if verRSA(ciphertext, signature, pk):
        return dechiffre(ciphertext, sk)
    raise Exception("Signature verification failed.")

In [131]:
# -- Exemple --
message = "Il commence à faire chaud ici..."
encode = HencS(message, pk, sk)
decode = HdecS(encode,  pk, sk)
print(f'Message décodé:\n{decode}')

Message décodé:
Il commence à faire chaud ici...


## 2. Certification de la clé RSA

Il est plus prudent d'obtenir un certificat de clé publique pour RSA !

**Exercice 17.** Comme dans le **TD6**, créez une requête en signature de certificat (*certificate signing request*) pour une clé RSA avec la librairie `Cryptography` de `Python` et enregistrez-la sur le disque avec l'extension `.csr`.

In [132]:
key = genRSA(1024)

csr = x509.CertificateSigningRequestBuilder().subject_name(x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, "FR"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, "PACA"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, "Valbonne"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, "Jawel Sacha Calvin")
])).sign(key, hashes.SHA256())

with open("example.csr", "wb") as file:
    file.write(csr.public_bytes(serialization.Encoding.PEM))

**Exercice 18.** Certifiez la requête en signature de la clé RSA en utilisant `OpenSSL` pour engendrer le certificat. Le certificat racine vous est fourni sous le nom `root_CA.crt` ainsi que la biclé correspondante `CA.pem`. Donnez ensuite la commande `OpenSSL` qui permet de vérifier la validité de ce certificat avec l'autorité racine fournie.

In [ ]:
openssl x509 -req -in example.csr -CA root_CA.crt -CAkey CA.pem -out example.crt -sha256
openssl verify -CAfile root_CA.crt example.crt

**Exercice 19.** Ecrivez une fonction `Python` `ReadRSACert` qui va lire le certificat à partir de son nom de fichier et retourner l'objet de clé publique correspondant:

In [ ]:
def ReadRSACert(file:str)->rsa.RSAPublicKey:
    with open(file, "rb") as f:
        data = f.read()
        certificate = x509.load_pem_x509_certificate(data)
        return certificate.public_key()

## 3. Utilisation dans un terminal

**Exercice 20.** Ecrivez deux scripts `Python` utilisables dans un terminal avec les paramètres, dans l'ordre:

- destinataire ;
- expéditeur ;
- texte pour `ecrire.py` ou fichier pour `lire.py`.

Le script `ecrire` prend en entrée le certificat de clé publique du destinataire, la clé privée de l'expéditeur, un petit texte au format `utf-8` et fournit le résultat du chiffrement hybride sur la sortie standard qui pourra être redirigé dans un fichier (`A2B.hyb` dans l'exemple ci-dessous).

Le script `lire` prend en entrée la clé privée du destinataire, le certificat de clé publique de l'expéditeur ainsi que le fichier résultat du chiffrement hybride et valide la vérification de l'expéditeur puis affiche le texte clair au format `utf-8`.

Vous fournirez les deux scripts dans l'archive du compte-rendu ainsi qu'un exemple d'utilisation en écrivant un petit message à l'utilisateur `bmartin` au moyen de son certificat de clé publique.

**Exemple**

`python ecrire.py bmartin.crt alice.pem "un petit exemple" > A2B.hyb`

`python lire.py bmartin.pem alice.crt A2B.hyb`